In [1]:
!pip install transformers

In [2]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

In [3]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', pad_token='<pad>')
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.5.attn.masked_bias', 'lm_head.weight', 'transformer.h.10.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.9.attn.masked_bias'

In [4]:
print(tokenizer.bos_token_id)
print(tokenizer.eos_token_id)
print(tokenizer.pad_token_id)
print('-' * 10)
print(tokenizer.decode(1))
print(tokenizer.decode(2))
print(tokenizer.decode(3))
print(tokenizer.decode(4))

1
1
3
----------
</s>
<usr>
<pad>
<sys>


In [5]:
import pandas as pd
import tqdm
import urllib.request

In [6]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')

In [7]:
len(train_data)

11823

In [8]:
# data 확인후 최대 길이까지 max_len 설정후 padding 처리 해야함
train_data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [9]:
train_data['Q'].map(len).max()

56

In [10]:
train_data['A'].map(len).max()

76

In [11]:
train_data['Q'].map(len).max() + train_data['A'].map(len).max() + 4

136

In [12]:
batch_size = 32

In [13]:
def get_chat_data():
  for question, answer in zip(train_data.Q.to_list(), train_data.A.to_list()):
    bos_token = [tokenizer.bos_token_id]
    eos_token = [tokenizer.eos_token_id]
    sent = tokenizer.encode('<usr>' + question + '<sys>' + answer)
    yield bos_token + sent + eos_token

- 불러온 batch_size 만큼 가장 긴 size를 padding화 해준다.

In [14]:
dataset = tf.data.Dataset.from_generator(get_chat_data, output_types=tf.int32)

In [15]:
dataset = dataset.padded_batch(batch_size=batch_size, padded_shapes=(None,), padding_values=tokenizer.pad_token_id)

In [16]:
for batch in dataset:
    print(batch)
    break

tf.Tensor(
[[    1     2  9349  7888   739  7318   376     4 12557  6824  9108  9028
   7098 25856     1     3     3     3     3     3     3     3     3     3
      3     3     3     3     3     3]
 [    1     2  9020  8263  7497 10192 11615  8210  8006     4 12422  8711
   9535  7483 12521     1     3     3     3     3     3     3     3     3
      3     3     3     3     3     3]
 [    1     2  9085  7597   395  8149 10624  7397 24224 13358  7182     4
  12079  8135 16899  9677  8234   389     1     3     3     3     3     3
      3     3     3     3     3     3]
 [    1     2  9085  7597   395  8149  9465 10624  7397 24224 13358  7182
      4 12079  8135 16899  9677  8234   389     1     3     3     3     3
      3     3     3     3     3     3]
 [    1     2  9943   422   418  9327  8702  7098     4  9847 16912 18328
   8671  7415  8263  8234   389     1     3     3     3     3     3     3
      3     3     3     3     3     3]
 [    1     2  9815   410 21249 10174  6824  8210  800

In [17]:
tokenizer.decode(batch[0])

'</s><usr> 12시 땡!<sys> 하루가 또 가네요.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [18]:
print(batch[0])

tf.Tensor(
[    1     2  9349  7888   739  7318   376     4 12557  6824  9108  9028
  7098 25856     1     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3], shape=(30,), dtype=int32)


In [19]:
print(tokenizer.encode('</s><usr> 12시 땡!<sys> 하루가 또 가네요.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'))

[1, 2, 9349, 7888, 739, 7318, 376, 4, 12557, 6824, 9108, 9028, 7098, 25856, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [20]:
adam = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)

In [21]:
steps = len(train_data) // batch_size + 1
print(steps)

370


- 예측시 다음 단어 예측
- 단어를 하나씩 밀어서
- 2개를 받아서 label이라는 단어를 하나씩 밀어준다.

In [25]:
from tqdm.notebook import tqdm

EPOCHS = 3

for epoch in range(EPOCHS):
  epoch_loss = 0

  for batch in tqdm(dataset, total=steps):
      with tf.GradientTape() as tape: # 자동 미분을 위한 context 생성 - parameter에 대한 loss의 gradient 계산
          result = model(batch, labels=batch) # batch data 전달,
          loss = result[0]
          batch_loss = tf.reduce_mean(loss)

      grads = tape.gradient(loss, model.trainable_variables)
      adam.apply_gradients(zip(grads, model.trainable_variables))
      epoch_loss += batch_loss / steps

  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, epoch_loss))

  0%|          | 0/370 [00:00<?, ?it/s]

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_1_device_/job:localhost/replica:0/task:0/device:CPU:0}} All elements in a batch must have the same rank as the padded shape for component0: expected rank 1 but got element with rank 2 [Op:IteratorGetNext] name: 

- batch 끌어다가 fine-tuning 해주는것.

In [ ]:
text = '오늘도 좋은 하루!'

In [ ]:
sent = '<usr>' + text + '<sys>'

In [ ]:
input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
input_ids = tf.convert_to_tensor([input_ids])

In [ ]:
output = model.generate(input_ids, max_length=50, early_stopping=True, eos_token_id=tokenizer.eos_token_id)

In [ ]:
decoded_sentence = tokenizer.decode(output[0].numpy().tolist())

In [ ]:
decoded_sentence.split('<sys> ')[1].replace('</s>', '')

In [ ]:
output = model.generate(input_ids, max_length=50, do_sample=True, top_k=10)
tokenizer.decode(output[0].numpy().tolist())

In [ ]:
def return_answer_by_chatbot(user_text):
  sent = '<usr>' + user_text + '<sys>'
  input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
  input_ids = tf.convert_to_tensor([input_ids])
  output = model.generate(input_ids, max_length=50, do_sample=True, top_k=20)
  sentence = tokenizer.decode(output[0].numpy().tolist())
  chatbot_response = sentence.split('<sys> ')[1].replace('</s>', '')
  return chatbot_response

In [ ]:
return_answer_by_chatbot('안녕! 반가워~')

In [ ]:
return_answer_by_chatbot('너는 누구야?')

In [ ]:
return_answer_by_chatbot('사랑해')

In [ ]:
return_answer_by_chatbot('나랑 영화보자')

In [ ]:
return_answer_by_chatbot('너무 심심한데 나랑 놀자')

In [ ]:
return_answer_by_chatbot('영화 해리포터 재밌어?')

In [ ]:
return_answer_by_chatbot('너 딥 러닝 잘해?')

In [ ]:
return_answer_by_chatbot('너 취했어?')

In [ ]:
return_answer_by_chatbot('커피 한 잔 할까?')